In [100]:
import docx
import pandas as pd
from datetime import datetime, date
from docx.shared import RGBColor, Pt, Inches
from docx.oxml import parse_xml
from docx.oxml.shared import OxmlElement
from docx.oxml.ns import qn, nsdecls
from docx.enum.table import WD_TABLE_ALIGNMENT

CONFIG THIS BEFORE RUN

In [101]:
Save_Dir = r"#"
Date1 = '01.02.23'
Date2 = 'January 2, 2023'

Data Preparation

In [102]:
Data_Repository = pd.read_excel(r"tblReport Query.xlsx", index_col=None)
Data_Repository = Data_Repository.rename(columns = {'Issue Number': 'Issue # Ref', 'Project Name': 'Audit Name', 'Management Action': 'Brief Description'})
Data_Repository['Target Date'] = pd.to_datetime(Data_Repository['Target Date']).dt.strftime('%m.%d.%y')
Data_Repository['Revised Target Date'] = pd.to_datetime(Data_Repository['Revised Target Date']).dt.strftime('%m.%d.%y')
Data_Repository['1st Revised Target Date'] = pd.to_datetime(Data_Repository['1st Revised Target Date']).dt.strftime('%m.%d.%y')
Data_Repository['2nd Revised Target Date'] = pd.to_datetime(Data_Repository['2nd Revised Target Date']).dt.strftime('%m.%d.%y')
Data_Repository['3rd Revised Target Date'] = pd.to_datetime(Data_Repository['3rd Revised Target Date']).dt.strftime('%m.%d.%y')
Data_Repository['Brief Description'] = Data_Repository['Brief Description'].str.lstrip()
today_o = date.today().strftime('%m.%d.%y')
today = datetime.strptime(today_o, "%m.%d.%y")

In [103]:
#Revised Date chronic order
for i in range(0, len(Data_Repository)):
    if pd.isnull(Data_Repository.iloc[i]['3rd Revised Target Date']) == False:
        list = [Data_Repository.iloc[i]['1st Revised Target Date'], Data_Repository.iloc[i]['2nd Revised Target Date'], Data_Repository.iloc[i]['3rd Revised Target Date']]
        list.sort(key = lambda date: datetime.strptime(date, '%m.%d.%y'))
        Data_Repository.loc[i, '1st Revised Target Date'] = Data_Repository.loc[i, '1st Revised Target Date'] = list[0]
        Data_Repository.loc[i, '2nd Revised Target Date'] = Data_Repository.loc[i, '2nd Revised Target Date'] = list[1]
        Data_Repository.loc[i, '3rd Revised Target Date'] = Data_Repository.loc[i, '3rd Revised Target Date'] = list[2]
    elif pd.isnull(Data_Repository.iloc[i]['2nd Revised Target Date']) == False:
        list = [Data_Repository.iloc[i]['1st Revised Target Date'], Data_Repository.iloc[i]['2nd Revised Target Date']]
        list.sort(key = lambda date: datetime.strptime(date, '%m.%d.%y'))
        Data_Repository.loc[i, '1st Revised Target Date'] = Data_Repository.loc[i, '1st Revised Target Date'] = list[0]
        Data_Repository.loc[i, '2nd Revised Target Date'] = Data_Repository.loc[i, '2nd Revised Target Date'] = list[1]


Body

In [104]:
def make_rows_bold(*rows):
    for row in rows:
        for cell in row.cells:
            for paragraph in cell.paragraphs:
                for run in paragraph.runs:
                    run.font.bold = True

#Row color
shading_elm_1 = parse_xml(r'<w:shd {} w:fill="#DEEAF6"/>'.format(nsdecls('w')))

def set_table_header_bg_color(cell):
    """
    set background shading for Header Rows
    """
    tblCell = cell._tc
    tblCellProperties = tblCell.get_or_add_tcPr()
    clShading = OxmlElement('w:shd')
    clShading.set(qn('w:fill'), "#D9D9D9")
    tblCellProperties.append(clShading)
    
def set_table_header_bg_color2(cell):
    """
    set background shading for Header Rows
    """
    tblCell = cell._tc
    tblCellProperties = tblCell.get_or_add_tcPr()
    clShading = OxmlElement('w:shd')
    clShading.set(qn('w:fill'), "#FFC7CE")
    tblCellProperties.append(clShading)

In [105]:
def body(To, cc, dept, Date1, Date2, evidence):
    doc = docx.Document()
    section = doc.sections[0]
    section.top_margin = Inches(0.62)
    section.bottom_margin = Inches(0.31)
    section.left_margin = Inches(0.75)
    section.right_margin = Inches(0.81)
    normal_style = doc.styles['Normal']
    normal_style.font.name = 'Arial'
    normal_style.font.size = Pt(10)
    normal_style.font.color.rgb = RGBColor(31, 73, 125)


    #Content
    main1 = '''
To: {}

cc: {}
    '''.format(To, cc)

    line1 = '''
Subject: {} Open Audit Issues / Action Plans Summary as of {}
    '''.format(dept, Date1)

    line2 = '''

Please find the Outstanding Audit Issues/Action Plans Summary as of {} (attached).

When actions have been completed, please provide the supporting evidence to close the action. Email evidence to {}.

Should you need to revise the target completion date, please send an email to Internal Audit DH, or designee, noting the revised date, reason for the delay and interim action to mitigate risk, as applicable.

Please note: Target Date extensions will need to be approved by the respective Division Head, via email.

    '''.format(Date2, evidence)

    parag = doc.add_paragraph(main1, 'Normal')
    parag.add_run(line1).font.color.rgb = RGBColor(0, 32, 96)
    parag.add_run(line2)

    #Data Table Preset
    excel = Data_Repository[Data_Repository['DepartmentResponsible'] == dept]
    n1 = 2
    if excel['3rd Revised Target Date'].notnull().any() == True:
        excel = excel[['Issue # Ref', 'Target Date', '1st Revised Target Date', '2nd Revised Target Date', '3rd Revised Target Date', 'Revised Target Date', 'Audit Name', 'Brief Description']]
    elif excel['2nd Revised Target Date'].notnull().any() == True:
        excel = excel[['Issue # Ref', 'Target Date', '1st Revised Target Date', '2nd Revised Target Date', 'Revised Target Date', 'Audit Name', 'Brief Description']]
    elif excel['1st Revised Target Date'].notnull().any() == True:
        excel = excel[['Issue # Ref', 'Target Date', '1st Revised Target Date', 'Revised Target Date', 'Audit Name', 'Brief Description']]
    elif excel['Revised Target Date'].notnull().any() == True:
        excel = excel[['Issue # Ref', 'Target Date', 'Revised Target Date', 'Audit Name', 'Brief Description']]
    else:
        excel = excel[['Issue # Ref', 'Target Date', 'Audit Name', 'Brief Description']]
    columns = excel.columns
    n2 = len(columns)
    excel = excel.fillna('')

    #Table Basic Format
    table = doc.add_table(rows = n1, cols = n2)
    table.allow_autofit = True
    table.alignment = WD_TABLE_ALIGNMENT.LEFT
    table.cell(0, 0).text = 'High-level Status' #First row text
    make_rows_bold(table.rows[0]) #First row Bold
    table.cell(0, 0).merge(table.rows[0].cells[-1]) #First row merge
    table.cell(0,0)._tc.get_or_add_tcPr().append(shading_elm_1) #First row color
    for col in range(len(columns)): #Column Names
        table.cell(1, col).text = columns[col]
        table.cell(1,col).paragraphs[0].paragraph_format.alignment = WD_TABLE_ALIGNMENT.CENTER
    for each_cell in table.rows[1].cells: #Second row color
        set_table_header_bg_color(each_cell)


    #Table Adding Contents
    for i, row in enumerate(excel.iloc[0:].itertuples()):
        table_row = table.add_row().cells
        for col in range(0, n2):
            table_row[col].text = str(row[col+1]) #+1 for ignore index
            table_row[col].paragraphs[0].paragraph_format.alignment = WD_TABLE_ALIGNMENT.LEFT

    #Expired Action Highlight
    if len(excel.columns) >= 5:
        for row in range(3, len(table.rows) + 1):
            if table.cell(row, -3).text != '':
                if datetime.strptime(table.cell(row, -3).text, "%m.%d.%y") < today:
                    set_table_header_bg_color2(table.rows[row -1].cells[-3]) 
            else:
                if datetime.strptime(table.cell(row -1, 1).text, "%m.%d.%y") < today:
                    set_table_header_bg_color2(table.rows[row - 1].cells[1])

    elif len(excel.columns) == 4:
        for row in range(3, len(table.rows) + 1):
            if datetime.strptime(table.cell(row, -3).text, "%m.%d.%y") < today:
                set_table_header_bg_color2(table.rows[row -1].cells[1])


    #Table margin setting
    table.rows[0].height = Inches(0.25)
    table.rows[1].height = Inches(0.3)

    for cell in table.columns[-1].cells:
        cell.width = Inches(7.35)

    #Change whole table text color
    for row in table.rows:
        for cell in row.cells:
            paragraphs = cell.paragraphs
            for paragraph in paragraphs:
                for run in paragraph.runs:
                    run.font.color.rgb = RGBColor(0,0,0)

    table.style = 'Table Grid'

    doc.save(Save_Dir + '\\' + dept + '_' + Date1 + '.docx')


Departments

In [106]:
deps = ['Accounting & Tax', 'Collections', 'Commercial Credit', 'Human Resources', 'Information Protection', 'Infrastructure Services', 'IT', 'Marketing', 'Product Development', 'Servicing & Loyalty']

In [107]:
#To = 0 cc = 1 evidence = 2
dict = {
    'Accounting & Tax': ['Elwin, Jinny', 'Gelashvili, Makhmud', 'Ted Jung'],

    'Collections': ['McKee,Temo', 'Leclerc, Ghjuvanni; Ishida, Neelam; Wehnert, Phil', 'Ted Jung'],

    'Commercial Credit': ['Rennoll, Annabelle', 'Gismondi, Sjors; Allegri, Siothrún; Xanthopoulos, Phuntsok', 'Ted Jung'],

    'Human Resources': ['Horn, Jacinto', 'Darnell, Xavier; Klasson, Hugo; Stojanović, Euadne', 'Ted Jung'],

    'Information Protection': ['Daniels, Dikla', 'Simon, Avrum; Farber, Ness; Mawuli, Murray', 'Ted Jung'],

    'Marketing': ['Kavanagh, Irina', 'Faucher, Boubacar', 'Ted Jung'],

    'Product Development': ['Romagna, Motecuhzoma', 'Michelakos, Ainoa', 'Ted Jung'],

    'Servicing & Loyalty': ['Jaskulski, Shalva', 'Kaur, Quanah; Weston, Girisha; Ansaldi, Tamatoa', 'Ted Jung']
}

Generator

In [108]:
for i in deps:
    body(dict[i][0], dict[i][1], i, Date1, Date2, dict[i][2]) #To, cc, dept, Date1, Date2, evidence